In [1]:
import re, nltk
import numpy as np

In [3]:
with open("resources/summa.txt", encoding="utf8") as myfile:
    file1 = " ".join(line.rstrip() for line in myfile)
    
with open("resources/summa2.txt", encoding="utf8") as myfile:
    file2 = " ".join(line.rstrip() for line in myfile)

In [4]:
def clean(data):
    text = re.sub(r"[^a-zA-Z]+", " ", data)
    text = re.sub(r"^.*?(?=(QUESTION))", "", text)
    text = re.sub(r"(?<=(FOR EVER BLESSED Amen)).+", '', text)
    text = re.sub(r"ON FATE", "QUESTION", text)
    
    return text


def chapters_split(data):
    chapters = []
    chap = []

    cleaned = clean(data).split(" ")

    for i, word in enumerate(cleaned):

        if (word == "QUESTION") or (i==len(cleaned)-1):
            chapters.append(' '.join(chap))
            chap = []

        else:
            chap.append(word)

    chapters = [chap for chap in chapters[1:]]
    
    return chapters

In [5]:
def text_split(data, stopWords):
    text = re.sub(r"[^\w]+", " ", data).split(" ")
    
    words = [word.lower() for word in text if word not in stopWords and len(word) > 1 and word[0] != "'"]
                
    return words


def shingles(file1, file2, k, stopWords):
    
    f1_words = text_split(file1, stopWords)
    f2_words = text_split(file2, stopWords)
    
    f1_shingles = set(tuple(f1_words[i:i+k]) for i in range(len(f1_words)-k+1))
    f2_shingles = set(tuple(f2_words[i:i+k]) for i in range(len(f2_words)-k+1))
    
    return f1_shingles, f2_shingles


def jaccard(f1, f2, k, stopWords):
    
    (f1_shingles, f2_shingles) = shingles(f1, f2, k, stopWords)
    
    intersection = f1_shingles.intersection(f2_shingles)
    union = f1_shingles.union(f2_shingles)
        
    return 1 - len(intersection)/len(union)


In [7]:
stopWords = set(nltk.corpus.stopwords.words('english'))

# 1a

In [8]:
f1 = clean(file1)
f2 = clean(file2)

jaccard(f1, f2, 4, stopWords)

0.9959429683339193

# 1b

In [9]:
chapters = chapters_split(file1)
# stopWords = set()

In [10]:
JDDS = []
for i in range(len(chapters)-1):
    JDDS.append([])
    for j in range(i+1,len(chapters)):
        JD = jaccard(chapters[i], chapters[j], 7, stopWords)
        JDDS[i].append(JD)
#         print("Chapters {} and {} distance: {}".format(i, j, JD))

# Task2